# Spectrogram Visualization
### Code from
https://gist.github.com/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e#file-music_genre_classification-ipynb

In [8]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import warnings

1 Physical GPUs, 1 Logical GPUs


## Extract Spectogram Features

In [9]:
filename = '../dataset/gitzan/features_30_sec.csv'
data = pd.read_csv(filename)

data.head()


filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.wav  661794          0.350088         0.088757  0.130228   
1  blues.00001.wav  661794          0.340914         0.094980  0.095948   
2  blues.00002.wav  661794          0.363637         0.085275  0.175570   
3  blues.00003.wav  661794          0.404785         0.093999  0.141093   
4  blues.00004.wav  661794          0.308526         0.087841  0.091529   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.002827             1784.165850          129774.064525   
1  0.002373             1530.176679          375850.073649   
2  0.002746             1552.811865          156467.643368   
3  0.006346             1070.106615          184355.942417   
4  0.002303             1835.004266          343399.939274   

   spectral_bandwidth_mean  spectral_bandwidth_var  ...  mfcc16_var  \
0              2002.449060            85882.761315  ...   52.420910   
1              2039.036516           213843.755497  ...   55.356403   
2              1747.702312            76254.192257  ...   40.598766   
3              1596.412872           166441.494769  ...   44.427753   
4              1748.172116            88445.209036  ...   86.099236   

   mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfcc19_mean  mfcc19_var  \
0    -1.690215   36.524071    -0.408979   41.597103    -2.303523   55.062923   
1    -0.731125   60.314529     0.295073   48.120598    -0.283518   51.106190   
2    -7.729093   47.639427    -1.816407   52.382141    -3.439720   46.639660   
3    -3.319597   50.206673     0.636965   37.319130    -0.619121   37.259739   
4    -5.454034   75.269707    -0.916874   53.613918    -4.404827   62.910812   

   mfcc20_mean  mfcc20_var  label  
0     1.221291   46.936035  blues  
1     0.531217   45.786282  blues  
2    -2.231258   30.573025  blues  
3    -3.407448   31.949339  blues  
4   -11.703234   55.195160  blues  

[5 rows x 60 columns]

In [10]:
data = data.drop(['filename'], axis=1)

In [11]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
len(X_train)

800

In [13]:
len(y_test)

200

In [14]:
X_train[10]

array([-0.13282213,  0.03889975, -0.49323156,  0.90855871,  0.01565838,
        0.05383062, -0.54645019, -0.09229221, -0.7537449 ,  0.09967402,
       -0.65195209,  0.07284395, -0.55812905,  0.21630701,  0.41521502,
        0.03605361,  0.74458663, -0.96354578,  0.99605779, -0.89923154,
        0.05229389, -0.85060822, -0.79997056, -0.94159839,  0.99854447,
       -1.05388111, -0.86530279, -0.32495822,  1.0400413 , -0.82028105,
       -0.82088474, -0.49674261,  1.01276048, -0.66121422, -1.26681391,
       -0.96239287,  0.52113547, -1.00457076, -0.68121963, -0.74077493,
        0.58250679, -0.8522493 , -1.72376833, -0.93049865,  0.61828532,
       -0.67539148, -1.26920387, -0.65238644,  0.74579709, -0.34752464,
        0.43364232, -0.7110238 ,  1.93138718, -0.71515927,  0.94356887,
       -0.68540097,  0.64228178, -0.54217946])

In [24]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train[1].shape[0],)))

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(10, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=20, batch_size=1)

Epoch 1/20
800/800 [==============================] - 4s 4ms/step - loss: 1.7773 - accuracy: 0.3625
Epoch 2/20
800/800 [==============================] - 3s 4ms/step - loss: 0.9518 - accuracy: 0.6699
Epoch 3/20
800/800 [==============================] - 3s 4ms/step - loss: 0.5622 - accuracy: 0.7892
Epoch 4/20
800/800 [==============================] - 4s 4ms/step - loss: 0.5322 - accuracy: 0.8005
Epoch 5/20
800/800 [==============================] - 4s 4ms/step - loss: 0.4296 - accuracy: 0.8415
Epoch 6/20
800/800 [==============================] - 3s 4ms/step - loss: 0.3124 - accuracy: 0.8862
Epoch 7/20
800/800 [==============================] - 3s 4ms/step - loss: 0.2219 - accuracy: 0.9277
Epoch 8/20
800/800 [==============================] - 3s 4ms/step - loss: 0.2402 - accuracy: 0.9139
Epoch 9/20
800/800 [==============================] - 3s 4ms/step - loss: 0.1355 - accuracy: 0.9626
Epoch 10/20
800/800 [==============================] - 3s 4ms/step - loss: 0.2881 - accuracy: 0.8956

In [29]:
test_loss, test_acc = model.evaluate(X_test,y_test, batch_size=1)
print(test_acc)

200/200 [==============================] - 1s 4ms/step - loss: 1.6992 - accuracy: 0.7300
0.7300000190734863
